In [ ]:
# Connect to drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


In [ ]:
# Move the directory to the csv files
%cd /content/drive/MyDrive/NLP/csv

/content/drive/.shortcut-targets-by-id/10jQIuTiIEwNUYyC2tiVRcsH1-ND-ng9R/NLP/csv


In [ ]:
ls

aclImdb/                           glove.6B.100d.txt
aclImdb_v1.tar.gz                  resume_classification_model/
cleaned_resume                     resume_dataset.csv
cleaned_resume.csv                 Resume_dataset_final.csv
cleaned_resume_dataset.csv         sample_da_job2.docx
cleaned_resume_dataset.gsheet      sample_da_job.docx
cleaned_resume_final.csv           test/
Data_analyst_sample_job_desc.gdoc  UpdatedResumeDataSet.csv


In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import  word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# packages

import pandas as pd
import numpy as nup 
import seaborn as sns
import re
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from nltk.stem import WordNetLemmatizer

In [ ]:
# Load required CSV files

df=pd.read_csv("cleaned_resume_final.csv")

In [ ]:
df['Category_no']=df.groupby('Category').ngroup()

In [ ]:
# Preprocessing the data
def preprocess_text(sentence_list):
  lines_list = []
  stop_words = set(stopwords.words('english'))
  lemmatizer = WordNetLemmatizer()

  for line in sentence_list:
      line = line.lower()
      words=[]
      for word in word_tokenize(line):
        if word.isalpha():
          if word not in stop_words:
            words.append(lemmatizer.lemmatize(word))
      lines_list.append(' '.join(words))
  
  return lines_list

In [ ]:
df.head()

,Unnamed: 0,Category,Resume,cleaned_resume,Category_no
0,0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,5
1,1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education detail may may data scientist data s...,5
2,2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...,5
3,3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...,5
4,4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...,5


In [ ]:
!pip install docx2txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3977 sha256=3eb98c68764256e49d50412b67812f271da9ab5b1a4a8e48966538c43312bcd9
  Stored in directory: /root/.cache/pip/wheels/40/75/01/e6c444034338bde9c7947d3467807f889123465c2371e77418
Successfully built docx2txt


In [ ]:
import docx2txt

In [ ]:
# Job description

job_description = docx2txt.process("sample_da_job2.docx")

In [ ]:
job_description_tokenised = preprocess_text([job_description])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

In [ ]:
df_data_science = df.loc[df['Category']=='Data Science']

In [ ]:
def get_similarity(resume_list):
  similarity_list = []
  for i in resume_list:
    combined_text = [job_description_tokenised[0], i]
    final_vector = vectorizer.fit_transform(combined_text)
    similarity_list.append(round(cosine_similarity(final_vector)[0][1]*100,2))

  return similarity_list

In [ ]:
df_data_science['Similarity_Percent'] = get_similarity(df_data_science['cleaned_resume'])

<ipython-input-19-3e0d86013ea6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_science['Similarity_Percent'] = get_similarity(df_data_science['cleaned_resume'])


In [ ]:
df_ds_sorted = df_data_science.sort_values(by='Similarity_Percent', ascending=False)

In [ ]:
df_ds_sorted['Predicted_Rank'] = range(1, len(df_ds_sorted) + 1)

In [ ]:
df_ds_sorted.shape

(40, 7)

In [ ]:
df_ds_sorted.head()

,Unnamed: 0,Category,Resume,cleaned_resume,Category_no,Similarity_Percent,Predicted_Rank
39,39,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,5,41.86,1
29,29,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,5,41.86,2
19,19,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,5,41.86,3
9,9,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,5,41.86,4
36,36,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,5,39.15,5


In [ ]:
import random

In [ ]:
manual_rank = range(1, len(df_ds_sorted) + 1)

In [ ]:
random.shuffle(manual_rank)

In [ ]:
print(manual_rank)

[31, 21, 16, 12, 18, 36, 30, 40, 4, 24, 28, 32, 22, 19, 8, 33, 27, 38, 17, 15, 37, 14, 20, 7, 13, 11, 23, 6, 25, 9, 29, 10, 34, 35, 3, 5, 26, 1, 39, 2]


In [ ]:
df_ds_sorted['Manual_Rank'] = manual_rank
df_ds_sorted

,Unnamed: 0,Category,Resume,cleaned_resume,Category_no,Similarity_Percent,Predicted_Rank,Manual_Rank
39,39,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,5,41.86,1,31
29,29,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,5,41.86,2,21
19,19,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,5,41.86,3,16
9,9,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,5,41.86,4,12
36,36,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,5,39.15,5,18
26,26,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,5,39.15,6,36
6,6,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,5,39.15,7,30
16,16,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,5,39.15,8,40
10,10,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,5,24.23,9,4
30,30,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,5,24.23,10,24


In [ ]:
!pip install rbo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import rbo

In [ ]:
rbo.RankingSimilarity(df_ds_sorted['Manual_Rank'].tolist(), df_ds_sorted['Predicted_Rank'].tolist()).rbo()

0.441641658022378